Load the (same) athletes dataset.

With your dataset use AutoML to find the best model (if it is no-code, submit screenshots).

Report any data insights from AutoML run.

What are the reported top 5 features?

What are the top 3 models per validation score when using

all features

only the top features (if you have to choose a number put in 3)

What are the top 3 models per speed when using

all features

only the top features (if you have to choose a number put in 3)

How does the top models compare to your previously developed model (assignments 1 and 2) in terms of validation score and speed?

Is your platform AutoML no-code/low-code/full-code and why?

Repeat 2 - 5 using H2O AutoML (https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.htmlLinks to an external site.). Submit code.

### This notebook uses H2O

I used ChatGTP 4o to help me understand key concepts and debug codes.

### Load the (same) athletes dataset.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Check current working directory
import os
os.getcwd()

'/content'

In [ ]:
!find /content/drive/MyDrive/ -name "athletes_cleaned.csv"

/content/drive/MyDrive/ML_OPS_Assignment3/athletes_cleaned.csv


In [3]:
import pandas as pd

athletes_df = pd.read_csv("/content/drive/MyDrive/ML_OPS_Assignment3/athletes_cleaned.csv")
athletes_df.head()

,athlete_id,name,region,affiliate,gender,age,height,weight,eat,train,background,howlong,total_lift
0,21269.0,Erik Acevedo,Southern California,CrossFit Training Yard,Male,30.0,71.0,200.0,I eat whatever is convenient|,I workout mostly at a CrossFit Affiliate|I inc...,College sports,1-2 years|,1110.0
1,21685.0,Richard Ablett,Africa,Cape CrossFit,Male,28.0,70.0,176.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|,No athletic background,2-4 years|,910.0
2,25464.0,Joe Abruzzo,North East,CrossFit Rapture,Male,35.0,68.0,225.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I rec...,Other,2-4 years|,1335.0
3,43767.0,Brigham Abbott,North Central,River North CrossFit,Male,36.0,71.0,199.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I hav...,College sports,1-2 years|,1354.0
4,55504.0,Jason Ackerman,North East,CrossFit Soulshine,Male,36.0,64.0,155.0,I eat strict Paleo|,"I workout mostly at home, work, or a tradition...",College sports,4+ years|,1225.0


In [4]:
athletes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30015 entries, 0 to 30014
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   athlete_id  30015 non-null  float64
 1   name        30015 non-null  object 
 2   region      30015 non-null  object 
 3   affiliate   29062 non-null  object 
 4   gender      30015 non-null  object 
 5   age         30015 non-null  float64
 6   height      30015 non-null  float64
 7   weight      30015 non-null  float64
 8   eat         30015 non-null  object 
 9   train       30015 non-null  object 
 10  background  30015 non-null  object 
 11  howlong     30015 non-null  object 
 12  total_lift  30015 non-null  float64
dtypes: float64(5), object(8)
memory usage: 3.0+ MB


### With your dataset use AutoML to find the best model

In [5]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
import h2o
h2o.init()

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpz2xjc5df
  JVM stdout: /tmp/tmpz2xjc5df/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpz2xjc5df/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_unknownUser_rp4rxr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [6]:
# They don’t carry real predictive power (they're IDs/names), they can confuse the model and lead to overfitting or poor generalization. Drop them before passing data to H2O.
athletes_df = athletes_df.drop(columns=["athlete_id", "name"])

In [7]:
# convert to H2OFrame
hf = h2o.H2OFrame(athletes_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
hf.types

{'region': 'enum',
 'affiliate': 'enum',
 'gender': 'enum',
 'age': 'int',
 'height': 'int',
 'weight': 'int',
 'eat': 'enum',
 'train': 'enum',
 'background': 'enum',
 'howlong': 'enum',
 'total_lift': 'int'}

In [9]:
# Define Features and Target
target = "total_lift"  # replace with your actual target
features = [col for col in hf.columns if col != target]

In [10]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(
    max_models=40, # limits the number of models trained
    seed=10, # ensures reproducibility
    exclude_algos=["DeepLearning", "StackedEnsemble"],  # can skip slow or unsupported models (e.g., DeepLearning in Colab CPU)
    sort_metric="RMSE"
)
aml.train(x=features, y=target, training_frame=hf)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_5_AutoML_1_20250720_10156


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    68                 68                          70734                  6            6            6             52            64            61.5294

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 17640.17258707093
RMSE: 132.8163114495766
MAE: 102.36987842799385
RMSLE: 0.16165603379005922
Mean Residual Deviance: 17640.17258707093

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 21527.421344035174
RMSE: 146.72225919755726
MAE: 113.1282136339952
RMSLE: 0.17617085828907816
Mean Residual Deviance: 21527.421344035174

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0          nan           nan           nan           nan           nan
loglikelihood           nan       0          nan           nan           nan           nan           nan
mae                     113.119   1.2706     111.239       114.679       112.674       113.481       113.523
mean_residual_deviance  21543.5   897.034    20159.4       22300.6       21188.9       22281.3       21787.4
mse                     21543.5   897.034    20159.4       22300.6       21188.9       22281.3       21787.4
r2                      0.720394  0.0107838  0.738259      0.715063      0.719722      0.709627      0.719299
residual_deviance       21543.5   897.034    20159.4       22300.6       21188.9       22281.3       21787.4
rmse                    146.751   3.07733    141.984       149.334       145.564       149.269       147.606
rmsle                   0.175846  0.0122222  0.165328      0.195364      0.166241      0.173549      0.178748

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2025-07-20 01:14:13  12.342 sec  0                  277.575          230.064         77048
    2025-07-20 01:14:13  12.567 sec  5                  211.004          171.775         44522.7
    2025-07-20 01:14:14  12.803 sec  10                 177.28           141.671         31428.2
    2025-07-20 01:14:14  13.039 sec  15                 158.985          125.2           25276.3
    2025-07-20 01:14:14  13.255 sec  20                 150.273          117.334         22582
    2025-07-20 01:14:14  13.487 sec  25                 145.364          112.969         21130.7
    2025-07-20 01:14:15  13.735 sec  30                 142.326          110.298         20256.8
    2025-07-20 01:14:15  13.957 sec  35                 140.155          108.411         19643.5
    2025-07-20 01:14:15  14.175 sec  40                 138.426          107.007         19161.9
    2025-07-20 01:14:15  14.405 sec  45                 137.145          105.914         18808.9
    2025-07-20 01:14:16  14.650 sec  50                 135.957          104.982         18484.3
    2025-07-20 01:14:16  14.903 sec  55                 134.954          104.166         18212.5
    2025-07-20 01:14:16  15.134 sec  60                 134.038          103.386         17966.1
    2025-07-20 01:14:16  15.360 sec  65                 133.244          102.726         17753.9
    2025-07-20 01:14:16  15.502 sec  68                 132.816          102.37          17640.2

Variable Importances: 
variable    relative_importance    s

H2O selected a Gradient Boosting Machine (GBM) as the best performer.

### Report any data insights from AutoML run.

Based on results above -

* **Best model:** H2O selected a **Gradient Boosting Machine (GBM)** as the best performer.
* **Validation RMSE (CV):** 146.72 — this is the **model’s error on unseen data**, a bit worse than previous best (\~139.25).
* **Training RMSE:** 132.82 — as expected, lower than validation (model fits training better).
* **Top Features:**

  1. `gender`
  2. `weight`
  3. `height`
  4. `train`
  5. `age`

In [11]:
# It is sorted by RMSE (Root Mean Squared Error) in ascending order (best-performing models first), because I set sort_metric="RMSE" in H2OAutoML.
lb = aml.leaderboard
lb.head(rows=10)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_5_AutoML_1_20250720_10156,146.722,21527.4,113.128,0.176171,21527.4
XGBoost_grid_1_AutoML_1_20250720_10156_model_9,146.753,21536.5,113.138,0.176369,21536.5
XGBoost_grid_1_AutoML_1_20250720_10156_model_14,146.833,21560,113.281,0.176439,21560
GBM_grid_1_AutoML_1_20250720_10156_model_7,146.841,21562.4,113.099,0.176363,21562.4
GBM_grid_1_AutoML_1_20250720_10156_model_6,146.926,21587.3,113.463,0.176485,21587.3
GBM_grid_1_AutoML_1_20250720_10156_model_3,146.972,21600.8,113.412,0.176679,21600.8
XGBoost_grid_1_AutoML_1_20250720_10156_model_7,147.037,21619.8,113.44,0.176494,21619.8
GBM_2_AutoML_1_20250720_10156,147.178,21661.4,113.441,0.17667,21661.4
XGBoost_grid_1_AutoML_1_20250720_10156_model_10,147.231,21677.1,113.607,0.176701,21677.1
XGBoost_grid_1_AutoML_1_20250720_10156_model_6,147.334,21707.3,113.566,0.176713,21707.3


### What are the reported top 5 features?

gender,  weight, height, train, age

In [16]:
# The top 5 features are calculated based on the best model only (aml.leader)
model = aml.leader
model.varimp(use_pandas=True)  # Only works if the model supports variable importance

,variable,relative_importance,scaled_importance,percentage
0,gender,4.077556e+09,1.000000,0.470273
1,weight,1.407311e+09,0.345136,0.162308
2,height,1.334746e+09,0.327340,0.153939
3,train,5.261837e+08,0.129044,0.060686
4,age,4.042733e+08,0.099146,0.046626
5,howlong,3.712116e+08,0.091038,0.042813
6,background,2.348804e+08,0.057603,0.027089
7,region,1.801239e+08,0.044174,0.020774
8,eat,8.969490e+07,0.021997,0.010345
9,affiliate,4.462719e+07,0.010945,0.005147


### Top 3 models per validation score using all features

In [17]:
# Get the top 3 models per validation score (lowest RMSE) from my H2O AutoML run using all features

# Access leaderboard
lb = aml.leaderboard.as_data_frame()

# Show top 3 models by RMSE (already sorted by RMSE by default)
top3_models = lb.head(3)
print("Top 3 Models by Validation RMSE (All Features):")
print(top3_models)

Top 3 Models by Validation RMSE (All Features):
                                          model_id        rmse           mse  \
0                    GBM_5_AutoML_1_20250720_10156  146.722259  21527.421344   
1   XGBoost_grid_1_AutoML_1_20250720_10156_model_9  146.753068  21536.463071   
2  XGBoost_grid_1_AutoML_1_20250720_10156_model_14  146.833376  21560.040335   

          mae     rmsle  mean_residual_deviance  
0  113.128214  0.176171            21527.421344  
1  113.138322  0.176369            21536.463071  
2  113.281065  0.176439            21560.040335  


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


### Top 3 models per validation score using only the top 3 features


In [18]:
# Select top 3 features + target
top3_features = ['gender', 'weight', 'height']
target = 'total_lift'
features_top3 = top3_features  # Explicit for clarity

# Subset H2OFrame to only needed columns
hf_top3 = hf[top3_features + [target]]

# Run AutoML on top 3 features
aml_top3 = H2OAutoML(
    max_models=40,
    seed=10,
    exclude_algos=["DeepLearning", "StackedEnsemble"],
    sort_metric="RMSE"
)
aml_top3.train(x=features_top3, y=target, training_frame=hf_top3)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_2_20250720_23914_model_6


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          20158                  5            5            5             17            32            27.38

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 30079.241213428842
RMSE: 173.43367958222197
MAE: 135.6455351161496
RMSLE: 0.20005713436593994
Mean Residual Deviance: 30079.241213428842

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 30582.293395826702
RMSE: 174.87793856237755
MAE: 136.7911905606015
RMSLE: 0.2017202469429303
Mean Residual Deviance: 30582.293395826702

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0           nan           nan           nan           nan           nan
loglikelihood           nan       0           nan           nan           nan           nan           nan
mae                     136.791   1.69196     135.754       138.373       134.376       137.341       138.11
mean_residual_deviance  30583.1   928.43      29753.7       31218.9       29406         31392.7       31144.2
mse                     30583.1   928.43      29753.7       31218.9       29406         31392.7       31144.2
r2                      0.603094  0.00931487  0.613689      0.601113      0.61103       0.590886      0.598749
residual_deviance       30583.1   928.43      29753.7       31218.9       29406         31392.7       31144.2
rmse                    174.864   2.66259     172.493       176.689       171.482       177.18        176.477
rmsle                   0.201496  0.0105737   0.194462      0.218085      0.191131      0.199183      0.204618

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2025-07-20 02:45:19  52.203 sec  0                  277.575          230.064         77048
    2025-07-20 02:45:19  52.324 sec  5                  217.28           176.464         47210.7
    2025-07-20 02:45:19  52.402 sec  10                 190.928          152.243         36453.5
    2025-07-20 02:45:19  52.480 sec  15                 180.481          142.38          32573.4
    2025-07-20 02:45:19  52.555 sec  20                 176.506          138.566         31154.4
    2025-07-20 02:45:19  52.630 sec  25                 174.898          137.014         30589.2
    2025-07-20 02:45:20  52.707 sec  30                 174.199          136.328         30345.2
    2025-07-20 02:45:20  52.785 sec  35                 173.88           136.031         30234.4
    2025-07-20 02:45:20  52.862 sec  40                 173.644          135.822         30152.4
    2025-07-20 02:45:20  52.949 sec  45                 173.515          135.717         30107.6
    2025-07-20 02:45:20  53.037 sec  50                 173.434          135.646         30079.2

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
gender      5.45887e+09            1                    0.800853
weight      1.21333e+09            0.222268             0.178004
height      1.44118e+08            0.0264007            0.021143

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.displ

In [19]:
# Show top 3 models by validation RMSE
lb_top3 = aml_top3.leaderboard.as_data_frame()
print("Top 3 Models by Validation RMSE (Top 3 Features):")
print(lb_top3.head(3))

Top 3 Models by Validation RMSE (Top 3 Features):
                                      model_id        rmse           mse  \
0   GBM_grid_1_AutoML_2_20250720_23914_model_6  174.877939  30582.293396   
1  GBM_grid_1_AutoML_2_20250720_23914_model_10  175.065538  30647.942607   
2  GBM_grid_1_AutoML_2_20250720_23914_model_11  175.115908  30665.581384   

          mae     rmsle  mean_residual_deviance  
0  136.791191  0.201720            30582.293396  
1  137.012938  0.201875            30647.942607  
2  137.048589  0.201990            30665.581384  


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Based on results above -

### **Best Model**

* **Type**: Gradient Boosting Machine (GBM)
* **Training RMSE (173.43)**: How well the model fits the training data. Lower = better fit.
* **Validation RMSE (174.88)**: How well the model performs on unseen data (via cross-validation). Lower = better generalization.

Since **Validation RMSE ≈ Training RMSE**, the model is **not overfitting** and generalizes **well** — though performance is weaker compared to using all features.
* **Interpretation**: Slight overfitting, but relatively close — model generalizes decently.


---

### **Top 3 Features (by importance)**

| Feature    | Contribution |
| ---------- | ------------ |
| **gender** | \~80.1%      |
| **weight** | \~17.8%      |
| **height** | \~2.1%       |

> **gender** dominates the predictive power in this reduced model.

---

### Key Insight:

* Performance dropped **from 139.25 RMSE (all features)** → **174.88 RMSE (top 3 features)**.
* Suggests that additional features beyond the top 3 meaningfully improve predictions.



**FLAML vs H2O**

* **FLAML (LightGBM)** outperformed **H2O (GBM)** on validation RMSE:

  * **FLAML RMSE**: **137.89**
  * **H2O RMSE**: **146.72**

* **Training RMSE** is slightly lower for both, suggesting **no major overfitting**.

* **Conclusion**:
  **FLAML** found a better-performing model (lower error)

  **FLAML** was also faster (60 sec vs H2O's longer runtime)

**Overall**, FLAML was **more efficient and more accurate** for this task. But H2O is really easy to install and run.
